In [1]:
import json
import re

data = []
with open('dump_small.jsonln', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        
print(f'Documents: {len(data)}')

Documents: 11225


## Functions to clean wikipedia's text.

In [2]:
def removeUrl(texto):
    # https://www.geeksforgeeks.org/python-check-url-string/
    pattern = r"""
        (?i)
        \b  
        (?:
            https?://
        |
            www
            \d{0,3}
            [.]
        |
            [a-z0-9.-]+
            [.]
            [a-z]{2,4}
            /
        )
        (?:
            [^\s()<>]+
        |
            (
            (?:
                [^\s()<>]+
            |
                (
                [^\s()<>]+
                )
            )*
            )
        )+
        (?:
            (
            (?:
                [^\s()<>]+
            |
                (
                [^\s()<>]+
                )
            )*
            )
        |
            [^\s`!()[]{};:'".,<>?«»“”‘’]
        )
    """
    repl = ''
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(repl, texto)

In [3]:
def removeWikipediaLinks(text):
    pattern = r'\[\[(?:[^|]*?\|)*?([^|]*?)\]\]'
    repl = r'\1'
    matcher = re.compile(pattern)
    return matcher.sub(repl, text)

In [4]:
def removeRef(text):
    return re.sub(r'<ref(?:.*)?>.*?(?:</ref>)?', '', text)

In [5]:
def removeMath(text):
    return re.sub(r'<math(?:.*)?>.*?(?:</math>)?', '', text)

In [6]:
def removeOtherTags(text):
    return re.sub(r'<.*?>(.*?)<.*?>', r'\1', text)

In [7]:
def removeHtmlFormat(text):
    return re.sub(r'\w+=".*"', '', text)

In [8]:
def removeOtherWikiFormat(text):
    return re.sub(r'\w+=.+', '', text)
removeOtherWikiFormat("</font>||<font color=\"black\">'''Campeonato Ga\u00facho '''</font>||<font color=\"black\">'''Posi\u00e7\u00e3o'''</font>\n|- bgcolor=ACE1AF\n|align=center|'''1927'''||align=center|'''{{BR-RSb}} 1\u00aa Divis\u00e3o'''||align=center|'''3\u00ba'''\n|- bgcolor=ACE1AF\n|align=center|'''1949'''||align=center|'''{{BR-RSb}} 1\u00aa Divis\u00e3o'''||align=center|'''3\u00ba'''\n|- bgcolor=ACE1AF\n|align=center|'''1952'''||align=center|'''{{BR-RSb}} 1\u00aa Divis\u00e3o'''||align=center|'''4\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1955'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''3\u00ba'''\n|- bgcolor=ACE1AF\n|align=center|'''1958'''||align=center|'''{{BR-RSb}} 1\u00aa Divis\u00e3o'''||align=center|'''3\u00ba'''\n|- bgcolor=ACE1AF\n|align=center|'''1960'''||align=center|'''{{BR-RSb}} 1\u00aa Divis\u00e3o'''||align=center|'''4\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1964'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''3\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1965'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''4\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1966'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''?'''\n|- bgcolor=FFCCCC\n|align=center|'''1967'''||align=center|'''{{BR-RSb}} 3\u00aa Divis\u00e3o'''||align=center|'''?'''\n|- bgcolor=FFCCCC\n|align=center|'''1968'''||align=center|'''{{BR-RSb}} 3\u00aa Divis\u00e3o'''||align=center|'''?'''\n|- bgcolor=ffdead\n|align=center|'''1969'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''?'''\n|- bgcolor=ffdead\n|align=center|'''1970'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''?'''\n|- bgcolor=ffdead\n|align=center|'''1978'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''5\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1979'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''Vice-campe\u00e3o'''\n|- bgcolor=ffdead\n|align=center|'''1980'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''8\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1981'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''8\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1982'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''6\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1983'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''4\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1984'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''3\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1985'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''5\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1986'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''7\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1987'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''6\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1988'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''15\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1989'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''?'''\n|- bgcolor=ffdead\n|align=center|'''1990'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''?'''\n|- bgcolor=ffdead\n|align=center|'''1991'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''?'''\n|- bgcolor=ffdead\n|align=center|'''1992'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''3\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1993'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''5\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1994'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''?'''\n|- bgcolor=ffdead\n|align=center|'''1995'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''5\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''1998'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''6\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''2005'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''18\u00b0'''\n|- bgcolor=ffdead\n|align=center|'''2006'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''23\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''2007'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''17\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''2008'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''17\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''2009'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''19\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''2010'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''18\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''2011'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''8\u00ba'''\n|- bgcolor=ffdead\n|align=center|'''2012'''||align=center|'''{{BR-RSb}} 2\u00aa Divis\u00e3o'''||align=center|'''16\u00ba'''\n|- bgcolor=FFCCCC\n|align=center|'''2013'''||align=center|'''{{BR-RSb}} 3\u00aa Divis\u00e3o'''||align=center|'''11\u00b0'''\n|- bgcolor=FFCCCC\n|align=center|'''2015'''||align=center|'''{{BR-RSb}} 3\u00aa Divis\u00e3o'''||align=center|'''9\u00b0'''\n|}\n\n{{col-2}}\n")

'</font>||<font \n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|- \n|\n|}\n\n{{col-2}}\n'

In [9]:
def removePipe(text):
    return re.sub('\|.*\n', '', text)

removePipe(".\n | align      = right\n | width1    = 200\n | width2    = 250\n\n | image1    = Satellite image of the Netherlands in May 2000.jpg\n | alt1      = Imagem de sat\u00e9lite\n\n | image2    = The_Netherlands_compared_to_sealevel.png\n | alt2      = Mapa\n}}")

'.\n   \n  \n  }}'

In [10]:
def removeQuotationMark(text):
    pattern = r"""(['"]+)(.*?)\1"""
    matcher = re.compile(pattern)
    return matcher.sub(r'\2', text)

In [11]:
def removeEqual(text):
    return text.replace('=',' ') 

In [12]:
def removeBraces(text):
    pattern = r"{.*?}"
    matcher = re.compile(pattern)
    return matcher.sub('', text)


In [13]:
def removeBrackets(text):
    pattern = r"\[.*?\]"
    matcher = re.compile(pattern)
    text = matcher.sub('', text)
    text = text.replace(']', '')
    return text.replace('[', '')

In [14]:
def removeHtmlComment(texto):
    pattern = r"<!--.*?-->"
    matcher = re.compile(pattern)
    return matcher.sub('', texto)

In [15]:
def removeAsterisk(texto):
    texto = texto.replace("*", "")
    return texto

In [16]:
def removePx(text):     
    return re.sub(r"(?:\d+x)?\d+px", ' ', text)

removePx("o problema.624pxDurante a adolescênci")

'o problema. Durante a adolescênci'

In [17]:
def removeDash(text):
    return re.sub(r'\s+[-–]\s+', ' ', text)

removeDash("Vista - Alegre Sport Clube Beira-Mar")

'Vista Alegre Sport Clube Beira-Mar'

In [18]:
def removeParentheses(text):
    text = text.replace('(', '')
    return text.replace(')', '')

In [19]:
def removeUpperO(text): # °
    return text.replace('\u00ba', '')


In [20]:
def removeDigits(text):
    return re.sub('\d+', '', text)

In [21]:
def removeWikipediaTemplates(text):
    conta = 0
    spans_proibidos = []
    for item in re.finditer(r'{{|}}', text):
        if item[0] == '{{':
            if conta == 0:
                inicio = item.span()[0]
            conta += 1
        else:
            conta -= 1
            if conta == 0:
                fim = item.span()[1]
                spans_proibidos.append((inicio, fim))

    clean_text = ''
    inicio = 0
    for span in spans_proibidos:
        fim, novo_inicio = span
        clean_text += text[inicio:fim]
        inicio = novo_inicio
    
    clean_text += text[inicio:]
    return clean_text



In [22]:
def clearText(text):
    funcs = [
        removeUrl,
        removeWikipediaLinks,
        removeRef,
        removeMath,
        removeOtherTags,
        removeHtmlFormat,
        removeOtherWikiFormat,
        removePipe,
        removeUpperO,
        removeQuotationMark,
        removeEqual,
        removeBraces,
        removeBrackets,
        removeHtmlComment,
        removeAsterisk,
        removePx,
        removeDash,
        removeParentheses,
        removeDigits,
        removeWikipediaTemplates
    ]
    x = text
    for func in funcs:
        x = func(x)
    return x

In [23]:
import json
with open('dump_small_clean.jsonln', 'w', encoding="utf8") as file:
    for i in range(len(data)):
        json.dump({"body": clearText(data[i]['body']), "title": data[i]["title"]}, file)
        file.write('\n')